In [1]:
%tensorflow_version 2.x

import tensorflow as tf
!pip install mitdeeplearning
import mitdeeplearning as mdl

import numpy as np
import os
import time

import functools
from IPython import display as ipythondisplay
from tqdm import tqdm

!apt-get install abcmidi timidity > /dev/null 2>&1

assert len(tf.config.list_physical_devices('GPU')) >0

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 25.6 MB/s 
  Created wheel for mitdeeplearning: filename=mitdeeplearning-0.2.0-py3-none-any.whl size=2115441 sha256=4e0ea4b903d7f2d0f3d577b5484745151aaea380fd0c03aabf62fef6c43ae8d3
  Stored in directory: /root/.cache/pip/wheels/9a/b9/4f/99b7c8c5c75355550b83e1fcfc02956fb40c35eb01e2262877
Successfully built mitdeeplearning


AssertionError: ignored

In [2]:
songs = mdl.lab1.load_training_data()

example_song = songs[0]
print("\n Example song:")
print(example_song)

Found 817 songs in text

 Example song:
X:1
T:Alexander's
Z: id:dc-hornpipe-1
M:C|
L:1/8
K:D Major
(3ABc|dAFA DFAd|fdcd FAdf|gfge fefd|(3efe (3dcB A2 (3ABc|!
dAFA DFAd|fdcd FAdf|gfge fefd|(3efe dc d2:|!
AG|FAdA FAdA|GBdB GBdB|Acec Acec|dfaf gecA|!
FAdA FAdA|GBdB GBdB|Aceg fefd|(3efe dc d2:|!


In [ ]:
mdl.lab1.play_song(example_song)

In [6]:
songs_joined = "\n\n".join(songs)

#find the all unique characters in the joined string
vocab = sorted(set(songs_joined))
print("There are", len(vocab), "unique characters in the dataset")

There are 83 unique characters in the dataset


In [7]:
### define a numerical representation of text ###

# create a mapping from character to unique index
#for example , to goet the index of the character "d"
# we can evaluate 'char2idx["d"]'.

char2idx = {u:i for i, u in enumerate(vocab)}

# Create a mapping from indices to characters. This is the inverse
# of char2idx and allows us to convert back from unique index to 
# the character in our vocabulary.

idx2char = np.array(vocab)

In [8]:
print('{')

for char, _ in zip(char2idx, range(20)):
  print('{:4s}: {:3d},'.format(repr(char), char2idx[char]))

print('....\n')

{
'\n':   0,
' ' :   1,
'!' :   2,
'"' :   3,
'#' :   4,
"'" :   5,
'(' :   6,
')' :   7,
',' :   8,
'-' :   9,
'.' :  10,
'/' :  11,
'0' :  12,
'1' :  13,
'2' :  14,
'3' :  15,
'4' :  16,
'5' :  17,
'6' :  18,
'7' :  19,
....



In [9]:
### vectorize the song string

def vectorize_string(string):
  vectorized_output = np.array([char2idx[char] for char in string])
  return vectorized_output

vectorized_songs = vectorize_string(songs_joined)
  

In [13]:
print('{} ---- characters mapped to int ----> {}'.format(repr(songs_joined[:10]), vectorized_songs))
## check the vecotrized output is numpy array
assert isinstance(vectorized_songs, np.ndarray), "returned result should be numpy array"


'X:1\nT:Alex' ---- characters mapped to int ----> [49 22 13 ... 22 82  2]


In [15]:
### Batch definition to create training examples ###

def get_batch(vectorized_songs, seq_length, batch_size):
  # the length of the vectorized songs string
  n = vectorized_songs.shape[0] -1
  # random;y choose the string indices for the examples in the training
  idx = np.random.choice(n-seq_length, batch_size)

  input_batch = [vectorized_songs[i: i+seq_length] for i in idx]

  output_batch = [vectorized_songs[i+1 : i+seq_length+1] for i in idx]

  x_batch = np.reshape(input_batch, [batch_size, seq_length])
  y_batch = np.reshape(output_batch, [batch_size, seq_length])

  return x_batch, y_batch

# Perform some simple tests to make sure your batch function is working

test_args = (vectorized_songs, 10, 2)
if not mdl.lab1.test_batch_func_types(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_shapes(get_batch, test_args) or \
   not mdl.lab1.test_batch_func_next_step(get_batch, test_args): 
   print("======\n[FAIL] could not pass tests")
else: 
   print("======\n[PASS] passed all tests!")

[PASS] test_batch_func_types
[PASS] test_batch_func_shapes
[PASS] test_batch_func_next_step
[PASS] passed all tests!


In [16]:
x_batch, y_batch = get_batch(vectorized_songs, seq_length=5, batch_size=1)

for i, (input_idx, target_idx) in enumerate(zip(np.squeeze(x_batch), np.squeeze(y_batch))):
  print("Step {:3d}".format(i))
  print("input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
  print(" expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))
  


Step   0
input: 15 ('3')
 expected output: 20 ('8')
Step   1
input: 20 ('8')
 expected output: 13 ('1')
Step   2
input: 13 ('1')
 expected output: 0 ('\n')
Step   3
input: 0 ('\n')
 expected output: 38 ('M')
Step   4
input: 38 ('M')
 expected output: 22 (':')


In [17]:
def LSTM(rnn_units):
  return tf.keras.layers.LSTM(
      rnn_units,
      return_sequences= True,
      recurrent_initializer ='glorot_uniform',
      recurrent_activation ='sigmoid',
      sateful = True,
  )

In [25]:
### Defining the RNN Model ###

'''TODO: Add LSTM and Dense layers to define the RNN model using the Sequential API.'''
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # Layer 1: Embedding layer to transform indices into dense vectors 
    #   of a fixed embedding size
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),

    # Layer 2: LSTM with `rnn_units` number of units. 
    # TODO: Call the LSTM function defined above to add this layer.
    LSTM(rnn_units), 
    # LSTM('''TODO'''),

    # Layer 3: Dense (fully-connected) layer that transforms the LSTM output
    #   into the vocabulary size. 
    # TODO: Add the Dense layer.
    tf.keras.layers.Dense(vocab_size)
    # '''TODO: DENSE LAYER HERE'''
  ])

  return model

# Build a simple model with default hyperparameters. You will get the 
#   chance to change these later.
model = build_model(len(vocab), embedding_dim=256, rnn_units=1024, batch_size=32)

TypeError: ignored